In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from looptools.loop import LOOP
from looptools.component import Component
from looptools.dimension import Dimension
from looptools.components import PIControllerComponent, DoubleIntegratorComponent, PIIControllerComponent
from looptools.loopmath import db_to_log2_gain, gain_for_crossover_frequency, add_transfer_function

In [ ]:
# Loop sample rate
sps = 80e6

# Define gain parameters
Kp_log2 = -10
Ki_log2 = -15
Kii_log2 = -20

# Double integrator extrapolation settings
extrapolate = (True, 1)

# Frequency range for Bode plot
frfr = np.logspace(np.log10(1e-6), np.log10(sps/2), int(1e5))
frfr = frfr[0:-1]

# Define controllers
pi = PIControllerComponent("P", sps, Kp_log2, -np.inf)
ii = DoubleIntegratorComponent("Double I", sps, Ki_log2, Kii_log2, extrapolate)
pii_1 = pi + ii
pii_2 = PIIControllerComponent("PII", sps, Kp_log2, Ki_log2, Kii_log2, extrapolate)

# Build composite loop
loop1 = LOOP(sps, [pii_1])
loop2 = LOOP(sps, [pii_2])

# Plot Bode diagrams
fig, axes = loop1.bode_plot(frfr, label='P + Double I')
loop2.bode_plot(frfr, axes=axes, label='PII', ls='--')
plt.show()

In [ ]:
# Plot Nyquist diagrams
fig, ax = loop1.nyquist_plot(frfr, label='P + Double I', logy=True, logx=True, alpha=0.5, critical_point=True)
loop2.nyquist_plot(frfr, ax=ax, label='PII', ls=':')
plt.show()

In [ ]:
# Test of loopmath::gain_for_crossover_frequency

sps = 80e6
frfr = np.logspace(np.log10(1e-6), np.log10(sps/2), int(1e5))
frfr = frfr[0:-1]

Kp_db = 20
f_cross = 1.0e3
Kp_log2 = db_to_log2_gain(Kp_db)
Kp = 2 ** Kp_log2

unit = Dimension(["cycle"], ["s", "rad"])

# === Additive Structure: P + I ===
Ki_log2_add = gain_for_crossover_frequency(Kp_log2, sps, f_cross, kind='I', structure='add')
Ki_add = 2 ** Ki_log2_add

P_add = Component("P", sps, np.array([Kp]), np.array([1.0]), unit=unit)
I_add = Component("I", sps, np.array([Ki_add]), np.array([1.0, -1.0]), unit=unit)

PI_add = P_add + I_add

loop_add = LOOP(sps)
loop_add.add_component(PI_add, loop_update=True)

# === Multiplicative Structure: P * (1 + I) ===
Ki_log2_mul = gain_for_crossover_frequency(Kp_log2, sps, f_cross, kind='I', structure='mul')
Ki_mul = 2 ** Ki_log2_mul

P_mul = Component("P", sps, np.array([Kp]), np.array([1.0]), unit=unit)
I_mul = Component("I", sps, np.array([Ki_mul]), np.array([1.0, -1.0]), unit=unit)
one = Component("One", sps, np.array([1.0]), np.array([1.0]), unit=unit)

one_plus_I = one + I_mul
PI_mul = P_mul * one_plus_I

loop_mul = LOOP(sps)
loop_mul.add_component(PI_mul, loop_update=True)

# === Plot Comparison ===
fig, axes = loop_add.bode_plot(frfr, which='all', label="P + I", dB=True)
loop_mul.bode_plot(frfr, which='all', label="P * (1 + I)", axes=axes, ls='--', dB=True)
plt.show()